In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa


In [2]:
TRAINING_DIR = 'F:/Xray_Gaus/train'
TEST_DIR='F:/Xray_Gaus/test'
VAL_DIR='F:/Xray_Gaus/val'


In [3]:
num_classes = 2
IMG_SIZE = 224
BATCH_SIZE = 32
input_shape = (224,224, 3)

learning_rate = 0.001
weight_decay = 0.0001
num_epochs = 10

In [4]:
from keras_preprocessing.image import ImageDataGenerator

# Training IDG
train_idg = ImageDataGenerator(
    rescale = 1./255, 
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1, 
    shear_range=0.1, 
    zoom_range=0.1,
    horizontal_flip=True)

# Training Gen
train_gen = train_idg.flow_from_directory(
    TRAINING_DIR,
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='binary',
    classes=['normal_175_3','COVID-19'],
    subset='training'
) 

Found 350 images belonging to 2 classes.


In [5]:
val_idg = ImageDataGenerator(
    rescale = 1./255, 
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1, 
    shear_range=0.1, 
    zoom_range=0.1,
    horizontal_flip=True)

# Training Gen
val_gen = val_idg.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='binary',
    classes=['normal_20_3','COVID-19'],
    subset='training'
)

Found 40 images belonging to 2 classes.


In [6]:
test_idg = ImageDataGenerator(rescale=1./255)

# Test Gen
test_gen = test_idg.flow_from_directory(
        TEST_DIR,
        target_size=(IMG_SIZE, IMG_SIZE),
        shuffle=True,
        class_mode='binary',
    classes=['normal_20_3','COVID-19'],
        #subset='validation'
)

Found 40 images belonging to 2 classes.


In [7]:
inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

from tensorflow.keras.optimizers import Adam

base_model = keras.applications.InceptionV3(
    weights = 'imagenet',
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False)

base_model.trainable = False

x = base_model.output
x = keras.layers.Flatten()(x)
x = keras.layers.Dropout(0.4)(x)
x = keras.layers.Dense(2048, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.4)(x)
x = keras.layers.Dense(2048, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(1, activation='sigmoid')(x)

# se acopla el modelo
model = keras.Model(base_model.input, outputs)

# congelar capas
for layer in base_model.layers:
    layer.trainable = False

# compilar el modelo.
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')]
        )

model.summary()

[0][0]     
__________________________________________________________________________________________________
mixed7 (Concatenate)            (None, 12, 12, 768)  0           activation_60[0][0]              
                                                                 activation_63[0][0]              
                                                                 activation_68[0][0]              
                                                                 activation_69[0][0]              
__________________________________________________________________________________________________
conv2d_72 (Conv2D)              (None, 12, 12, 192)  147456      mixed7[0][0]                     
__________________________________________________________________________________________________
batch_normalization_72 (BatchNo (None, 12, 12, 192)  576         conv2d_72[0][0]                  
_________________________________________________________________________________________________

In [8]:
STEP_SIZE_TRAIN = train_gen.n // train_gen.batch_size
STEP_SIZE_VALID = val_gen.n // val_gen.batch_size

history=model.fit(x = train_gen,
          steps_per_epoch = STEP_SIZE_TRAIN,
          validation_data = val_gen,
          validation_steps = STEP_SIZE_VALID,
          epochs = 5)

Epoch 1/5
10/10 [==============================] - 33s 3s/step - loss: 1.0726 - accuracy: 0.6941 - precision: 0.6789 - recall: 0.6773 - val_loss: 82.2048 - val_accuracy: 0.4688 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/5
10/10 [==============================] - 19s 2s/step - loss: 1.1890 - accuracy: 0.8093 - precision: 0.8613 - recall: 0.7630 - val_loss: 6.8395 - val_accuracy: 0.7812 - val_precision: 0.9000 - val_recall: 0.6000
Epoch 3/5
10/10 [==============================] - 20s 2s/step - loss: 0.3924 - accuracy: 0.9059 - precision: 0.9383 - recall: 0.8620 - val_loss: 17.5693 - val_accuracy: 0.6250 - val_precision: 1.0000 - val_recall: 0.2000
Epoch 4/5
10/10 [==============================] - 22s 2s/step - loss: 0.2265 - accuracy: 0.9441 - precision: 0.9498 - recall: 0.9359 - val_loss: 3.5487 - val_accuracy: 0.7500 - val_precision: 1.0000 - val_recall: 0.5000
Epoch 5/5
10/10 [==============================] - 20s 2s/step - loss: 0.2109 - accuracy: 0.9201 - precisi

In [9]:
model.save('F:/X_ray_Models/IncV3_xray_model')

history.history

INFO:tensorflow:Assets written to: F:/X_ray_Models/IncV3_xray_model\assets


{'loss': [1.1137995719909668,
  1.0797934532165527,
  0.4633556008338928,
  0.2702917754650116,
  0.22272901237010956],
 'accuracy': [0.7610062956809998,
  0.8207547068595886,
  0.9056603908538818,
  0.9402515888214111,
  0.9276729822158813],
 'precision': [0.7564102411270142,
  0.846666693687439,
  0.9189189076423645,
  0.9548386931419373,
  0.9235668778419495],
 'recall': [0.7564102411270142,
  0.7888198494911194,
  0.8831169009208679,
  0.925000011920929,
  0.9294871687889099],
 'val_loss': [82.20478057861328,
  6.8395304679870605,
  17.569278717041016,
  3.5487112998962402,
  4.378469944000244],
 'val_accuracy': [0.46875, 0.78125, 0.625, 0.75, 0.84375],
 'val_precision': [0.0, 0.8999999761581421, 1.0, 1.0, 1.0],
 'val_recall': [0.0,
  0.6000000238418579,
  0.20000000298023224,
  0.5,
  0.6428571343421936]}

In [10]:
results = model.evaluate(test_gen, batch_size=10)

2/2 [==============================] - 2s 344ms/step - loss: 3.3259 - accuracy: 0.8250 - precision: 1.0000 - recall: 0.6500


In [11]:
y_pred = model.predict(test_gen, verbose=1)

#y_pred = np.argmax(y_pred, axis = 1)


for i in range(len(y_pred)):
    if y_pred[i]>=0.5:
        y_pred[i]=1
    else:
        y_pred[i]=0

2/2 [==============================] - 3s 265ms/step


In [12]:
y_true=test_gen.labels

In [13]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           0       0.48      0.65      0.55        20
           1       0.46      0.30      0.36        20

    accuracy                           0.48        40
   macro avg       0.47      0.47      0.46        40
weighted avg       0.47      0.47      0.46        40



In [14]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true, y_pred)

array([[13,  7],
       [14,  6]], dtype=int64)